In [1]:
#!pip install flask
#!pip install nest-asyncio
#!pip install flask dill numpy nest_asyncio scikit-learn

In [2]:
import numpy as np
import pandas as pd
from flask import Flask, request, render_template
import dill
import nest_asyncio
import traceback

In [3]:
try:
    with open("mdl.pkl", "rb") as file:
        model = dill.load(file)
    print("Model berhasil dimuat!")
    # Lakukan tes sederhana
    test_input = [[1, 2, 3, 4, 5, 6, 7, 8]]  # Sesuaikan dengan format input model Anda
    prediction = model.predict(test_input)
    print("Prediksi percobaan:", prediction)
except Exception as e:
    print(f"Gagal memuat model: {e}")
    import traceback
    print(traceback.format_exc())

Model berhasil dimuat!
Prediksi percobaan: [1]


In [4]:
# Terapkan nest_asyncio
nest_asyncio.apply()

# Buat aplikasi Flask
app = Flask(__name__)

# Global variables untuk model dan dataset
model = None
df = None

# Fungsi untuk memuat model
def load_model():
    global model
    try:
        with open("mdl.pkl", "rb") as file:
            model = dill.load(file)
        print("Model berhasil dimuat!")
    except FileNotFoundError:
        print("Model file 'mdl.pkl' tidak ditemukan. Pastikan file ada di direktori yang benar.")
        raise
    except Exception as e:
        print(f"Gagal memuat model: {e}")
        print(traceback.format_exc())
        raise

# Fungsi untuk memuat dataset
def load_dataset():
    global df
    try:
        df = pd.read_csv("../Dataset/Kontekstual DM/final_processed_data.csv")
        print("Dataset berhasil dimuat!")
    except FileNotFoundError:
        print("Dataset file tidak ditemukan. Pastikan file ada di direktori yang benar.")
        raise
    except Exception as e:
        print(f"Gagal memuat dataset: {e}")
        print(traceback.format_exc())
        raise

In [5]:
# Mapping kelas status kunjungan
status_kunjungan_mapping = {
    0: 'Berobat jalan',
    1: 'Rujuk lanjut', 
    2: 'Sembuh',
    3: 'Kunjungan sehat',
    4: 'Lain-lain/Pulang paksa/Meninggal'
}

In [6]:
# Route untuk halaman utama
@app.route("/")
def home():
    # Ambil data untuk dropdown dari dataset
    poliklinik_list = df['Jenis_Poliklinik'].unique()
    diagnosis_list = df['Nama_Diagnosis'].unique()
    provinsi_list = df['Provinsi_faskes_tujuan_rujukan'].unique()
    kepemilikan_list = df['Kepemilikan_faskes'].unique()
    jenis_faskes_list = df['Jenis_faskes'].unique()
    tipe_faskes_list = df['Tipe_faskes'].unique()
    poli_faskes_list = df['Poli_faskes'].unique()
    jenis_kunjungan_list = df['Jenis_Kunjungan'].unique()

    # Kirim data ke template
    return render_template("index.html", 
                           poliklinik_list=poliklinik_list,
                           diagnosis_list=diagnosis_list,
                           provinsi_list=provinsi_list,
                           kepemilikan_list=kepemilikan_list,
                           jenis_faskes_list=jenis_faskes_list,
                           tipe_faskes_list=tipe_faskes_list,
                           poli_faskes_list=poli_faskes_list,
                           jenis_kunjungan_list=jenis_kunjungan_list)


In [7]:
@app.route("/predict", methods=["POST"])
def predict():
    try:
        # Ambil input dari form
        jenis_poliklinik = request.form.get('Jenis_Poliklinik')
        nama_diagnosis = request.form.get('Nama_Diagnosis')
        nama_provinsi = request.form.get('Provinsi_faskes_tujuan_rujukan')
        kepemilikan_faskes = request.form.get('Kepemilikan_faskes')
        jenis_faskes = request.form.get('Jenis_faskes')
        tipe_faskes = request.form.get('Tipe_faskes')
        poli_faskes = request.form.get('Poli_faskes')
        jenis_kunjungan = request.form.get('Jenis_Kunjungan')

        # Validasi input
        if not all([
            jenis_poliklinik, nama_diagnosis, nama_provinsi, kepemilikan_faskes, 
            jenis_faskes, tipe_faskes, poli_faskes, jenis_kunjungan 
        ]):
            return render_template("index.html", 
                                   prediction_text="Semua kolom harus diisi!",
                                   poliklinik_list=df['Jenis_Poliklinik'].unique(),
                                   diagnosis_list=df['Nama_Diagnosis'].unique(),
                                   provinsi_list=df['Provinsi_faskes_tujuan_rujukan'].unique(),
                                   kepemilikan_list=df['Kepemilikan_faskes'].unique(),
                                   jenis_faskes_list=df['Jenis_faskes'].unique(),
                                   tipe_faskes_list=df['Tipe_faskes'].unique(),
                                   poli_faskes_list=df['Poli_faskes'].unique(),
                                   jenis_kunjungan_list=df['Jenis_Kunjungan'].unique())

        # Mapping input ke kode numerik
        try:
            kode_poliklinik = df.loc[df['Jenis_Poliklinik'] == jenis_poliklinik, 'Kode_Jenis_Poliklinik'].values[0]
            kode_diagnosis = df.loc[df['Nama_Diagnosis'] == nama_diagnosis, 'Kode_Diagnosis'].values[0]
            kode_provinsi = df.loc[df['Provinsi_faskes_tujuan_rujukan'] == nama_provinsi, 'Kode_Provinsi_faskes_tujuan_rujukan'].values[0]
            kode_kepemilikan = df.loc[df['Kepemilikan_faskes'] == kepemilikan_faskes, 'Kode_Kepemilikan_faskes'].values[0]
            kode_jenis_faskes = df.loc[df['Jenis_faskes'] == jenis_faskes, 'Kode_Jenis_faskes'].values[0]
            kode_tipe_faskes = df.loc[df['Tipe_faskes'] == tipe_faskes, 'Kode_Tipe_faskes'].values[0]
            kode_poli_faskes = df.loc[df['Poli_faskes'] == poli_faskes, 'Kode_Poli_faskes'].values[0]
            kode_jenis_kunjungan = df.loc[df['Jenis_Kunjungan'] == jenis_kunjungan, 'Kode_Jenis_Kunjungan'].values[0]
        except IndexError:
            return render_template("index.html", 
                                   prediction_text="Input tidak valid. Pastikan data sesuai dengan dataset.",
                                   poliklinik_list=df['Jenis_Poliklinik'].unique(),
                                   diagnosis_list=df['Nama_Diagnosis'].unique(),
                                   provinsi_list=df['Provinsi_faskes_tujuan_rujukan'].unique(),
                                   kepemilikan_list=df['Kepemilikan_faskes'].unique(),
                                   jenis_faskes_list=df['Jenis_faskes'].unique(),
                                   tipe_faskes_list=df['Tipe_faskes'].unique(),
                                   poli_faskes_list=df['Poli_faskes'].unique(),
                                   jenis_kunjungan_list=df['Jenis_Kunjungan'].unique())
            
        # Membuat array fitur untuk prediksi
        features = np.array([[
            kode_poliklinik, kode_diagnosis, kode_provinsi, kode_kepemilikan, 
            kode_jenis_faskes, kode_tipe_faskes, kode_poli_faskes, kode_jenis_kunjungan
        ]])

        # Debugging: Print fitur untuk memastikan
        print("Features for prediction:", features)

        # Melakukan prediksi
        prediction = model.predict(features)[0]
        print("Prediction result:", prediction)

        # Mapping hasil prediksi ke label status kunjungan
        prediction_label = status_kunjungan_mapping.get(prediction, "Unknown")

        # Menampilkan hasil prediksi
        return render_template("index.html", 
                               prediction_text=f"Status kunjungan pasien adalah: {prediction_label}",
                               selected_features={
                                   "Jenis Poliklinik": jenis_poliklinik,
                                   "Nama Diagnosis": nama_diagnosis,
                                   "Provinsi Faskes Tujuan Rujukan": nama_provinsi,
                                   "Kepemilikan Faskes": kepemilikan_faskes,
                                   "Jenis Faskes": jenis_faskes,
                                   "Tipe Faskes": tipe_faskes,
                                   "Poli Faskes": poli_faskes,
                                   "Jenis Kunjungan": jenis_kunjungan
                               },
                               poliklinik_list=df['Jenis_Poliklinik'].unique(),
                               diagnosis_list=df['Nama_Diagnosis'].unique(),
                               provinsi_list=df['Provinsi_faskes_tujuan_rujukan'].unique(),
                               kepemilikan_list=df['Kepemilikan_faskes'].unique(),
                               jenis_faskes_list=df['Jenis_faskes'].unique(),
                               tipe_faskes_list=df['Tipe_faskes'].unique(),
                               poli_faskes_list=df['Poli_faskes'].unique(),
                               jenis_kunjungan_list=df['Jenis_Kunjungan'].unique())
    except Exception as e:
        # Debugging: Log kesalahan
        print("Error in predict route:", e)
        return render_template("index.html", 
                               prediction_text=f"Terjadi kesalahan: {str(e)}",
                               poliklinik_list=df['Jenis_Poliklinik'].unique(),
                               diagnosis_list=df['Nama_Diagnosis'].unique(),
                               provinsi_list=df['Provinsi_faskes_tujuan_rujukan'].unique(),
                               kepemilikan_list=df['Kepemilikan_faskes'].unique(),
                               jenis_faskes_list=df['Jenis_faskes'].unique(),
                               tipe_faskes_list=df['Tipe_faskes'].unique(),
                               poli_faskes_list=df['Poli_faskes'].unique(),
                               jenis_kunjungan_list=df['Jenis_Kunjungan'].unique())


In [ ]:
# Muat model dan dataset sebelum menjalankan server
load_model()
load_dataset()

# Jalankan aplikasi
if __name__ == "__main__":
    try:
        app.run(debug=True, host='0.0.0.0', port=5000, use_reloader=False)
    except Exception as e:
        print(f"Gagal menjalankan server: {e}")
        print(traceback.format_exc())

Model berhasil dimuat!
Dataset berhasil dimuat!
 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.100.37:5000
Press CTRL+C to quit
192.168.100.37 - - [26/Dec/2024 11:24:50] "GET / HTTP/1.1" 200 -
192.168.100.37 - - [26/Dec/2024 11:25:18] "POST /predict HTTP/1.1" 200 -


Features for prediction: [[  1 343  98  98   1   5  98   1]]
Prediction result: 4


192.168.100.37 - - [26/Dec/2024 11:26:34] "POST /predict HTTP/1.1" 200 -


Features for prediction: [[ 14 656  21   3   2  14  49   1]]
Prediction result: 1
